In [1]:
import torch
import os
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pickle
from detection_datasets import *
with open('anchors_VOC0712trainval.pickle', 'rb') as handle:
    anchors = pickle.load(handle)

transforms = A.Compose([
    A.Resize(width=224, height=224),
    A.VerticalFlip(p=1.0),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc'))

device = torch.device('cuda:0')
dtype=torch.float32

C:\Vstanovlene\Anaconda Distribution\envs\new_gpu_env\lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# train_set = VOCDatasetV8(devkit_path = '../../datasets/VOCdevkit/', scales=[80, 40, 20], transforms=transforms, device=device, dtype=dtype)
train_set = VOCDatasetV2(devkit_path = '../../datasets/VOCdevkit/', scales=[20], anchors=anchors, transforms=transforms, device=device, dtype=dtype)

True ../../datasets/VOCdevkit/VOC2007\ImageSets\Main\trainval.txt
True ../../datasets/VOCdevkit/VOC2012\ImageSets\Main\trainval.txt


In [3]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=False)

In [4]:
from torch.profiler import profile, ProfilerActivity

In [7]:
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-5/inference_at_640'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=False) as prof:
    for i, img in enumerate(train_loader):
        del img
        prof.step()
        if i == 4:
            break

In [8]:
print(prof.key_averages(group_by_input_shape=True).table(row_limit=-1))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -----------------------------------------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls                                     Input Shapes  Total KFLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -----------------------------------------------  ------------  
                                          Profi

In [ ]:
print(train_set.object_placed, train_set.object_not_placed, train_set.object_placed + train_set.object_not_placed, train_set.total)